### Test tools for the ambient AI agent team experiment

In [1]:
import dotenv
dotenv.load_dotenv()

import os
import requests

from datetime import datetime
import time

from slack_sdk import WebClient

In [2]:
webhook_url = os.getenv("SLACK_WEBHOOK_URL")
channel_id = os.getenv("SLACK_CHANNEL_ID")
client = WebClient(token=os.getenv("SLACK_BOT_TOKEN"))

def send_slack_message(webhook_url, message):
    payload = {"text": message,}
    response = requests.post(webhook_url, json=payload)
    if response.status_code != 200:
        raise ValueError(f"Request to Slack returned an error {response.status_code}: {response.text}")

In [3]:
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
user_message = f"Right now it is {current_time}\nAll good?"

send_slack_message(webhook_url, user_message)

We can send a message to slack with a button to click.

In [4]:
payload = {
    "text": "MESSAGE HERE. HOW TO PROCEED?",
    "blocks" : [
        {
            "type": "actions",
            "elements": [
                {
                    "type": "button",
                    "text": {"type": "plain_text", "text": "Yes"},
                    "value": "yes",
                    "action_id": "button_click_yes"
                },
                {
                    "type": "button",
                    "text": {"type": "plain_text", "text": "No"},
                    "value": "no",
                    "action_id": "button_click_no"
                }
            ]
        }
    ]
}

Send with polling to wait for response.

In [7]:
def send_message_and_get_ts(message):
    response = client.chat_postMessage(channel=channel_id, text=message)
    return response["ts"]

def poll_for_feedback(channel_id, thread_ts, timeout=60):

    start_time = time.time()

    while time.time() - start_time < timeout:

        response = client.conversations_replies(channel=channel_id, ts=thread_ts)
        messages = response.get("messages", [])
        
        for msg in messages:
            # Skip the original message (which has the same ts as thread_ts)
            if msg.get("ts") == thread_ts:
                continue
                
            # If we found any reply message that's not the original, return it
            return msg.get("text")
            
        # No reply found yet, wait and try again
        time.sleep(5)
        
    return None

In [8]:
# Example usage:
message_text = "Hey from another world man!"
thread_ts = send_message_and_get_ts(message_text)
print("Waiting for user feedback...")

user_feedback = poll_for_feedback(channel_id, thread_ts)
print("Final user feedback:", user_feedback)

Waiting for user feedback...
Final user feedback: yelllow man


#### Putting everything together for 1 Slack Function

In [10]:
def slack_communication(message, require_feedback=False, timeout=60):
    """
    Send a message to Slack and optionally wait for human feedback.
    
    Args:
        message (str): The message to send to Slack
        require_feedback (bool): Whether to wait for human feedback
        timeout (int): Maximum time to wait for feedback in seconds
        
    Returns:
        str: Human feedback if require_feedback=True, otherwise confirmation message
    """
    # Send the message
    try:
        if require_feedback:
            # Use the client method when we need to get a response
            thread_ts = send_message_and_get_ts(message)
            print(f"Message sent, waiting for feedback (timeout: {timeout}s)...")
            
            # Wait for feedback
            feedback = poll_for_feedback(channel_id, thread_ts, timeout)
            
            if feedback:
                return feedback
            else:
                return "No feedback received within the timeout period."
        else:
            # Simple message, no feedback needed
            # Can use either webhook or client, using client for consistency
            response = client.chat_postMessage(channel=channel_id, text=message)
            return f"Message posted successfully: {message[:50]}..." if len(message) > 50 else message
        
    except Exception as e:
        return f"Error sending message: {str(e)}"

In [11]:
# Example usage:
# Just send a notification
result = slack_communication("This is a notification message, no response needed.")
print(result)

Message posted successfully: This is a notification message, no response needed...


In [12]:
# Send a message and wait for feedback
feedback = slack_communication("Please respond to this message. What do you think?", require_feedback=True)
print(f"Received feedback: {feedback}")

Message sent, waiting for feedback (timeout: 60s)...
Received feedback: heck yeah bro
